In [17]:
import numpy as np
from ray.rllib.policy.sample_batch import SampleBatch
import functions as f 
obs_shape = (4,)
num_steps = 5
obs = np.random.rand(num_steps, *obs_shape)
actions = np.random.rand(num_steps, 1)
import torch
train_batch = SampleBatch({"obs": torch.tensor(obs), "actions": torch.tensor(actions)})


In [18]:
train_batch["obs"]

tensor([[0.7717, 0.7555, 0.4162, 0.8083],
        [0.6699, 0.3030, 0.8442, 0.6400],
        [0.4124, 0.3183, 0.8200, 0.7787],
        [0.2755, 0.8202, 0.6806, 0.6561],
        [0.7704, 0.8374, 0.4464, 0.8358]], dtype=torch.float64)

In [19]:
#create a random torch model that takes the observations and outputs the logits
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist
import torch.distributions.normal as normal
import torch.distributions.multivariate_normal as multivariate_normal
import torch.distributions.kl as kl
import torch.distributions.bernoulli as bernoulli

class Model(nn.Module):
    def __init__(self, obs_shape, action_shape):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(obs_shape[0], 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_shape[0])
        
    def forward(self, x):
        x = x.float()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [20]:
model = Model((5,4), (1,))

In [21]:

def loss1(train_batch):
    # get the observations and actions
    obs, actions = train_batch["obs"], train_batch["actions"]
    loss= 0 
    # get the logits and the state of the model
    logits, _ = model(obs)
    
    # calculate the mean of L_T and L_S over the training batch
    L_S = 0


    #get a bunch of normal distribution around 
    dist = torch.distributions.Normal(obs, 0.1 )

    around_obs = dist.sample()

    logits_around, _ = model( around_obs)



    L_S = 0
    L_T = 0

    for i in range (len(train_batch["actions"])):


        # get the loss of the state around the observations
        L_S += torch.mean(abs(logits[i]-logits_around[i]))

        # get the loss of the actions around the observations
        if(i>0):
            L_T +=  f.action_dist(actions[i],actions[i-1])
        
    L_S = L_S / len(train_batch["actions"])
    L_T = L_T / len(train_batch["actions"])
    
    # add the loss of the state around the observations to the loss
    loss +=  L_S
    loss += L_T
    return loss

In [22]:
import torch
import torch.nn.functional as F

def loss2(train_batch: SampleBatch):
    loss = 0 
    # get the observations and actions
    obs, actions = train_batch["obs"], train_batch["actions"]
    
    # get the logits and the state of the model
    logits, _ = model(obs)
    
    # calculate the mean of L_T and L_S over the training batch
    L_S = 0

    # get a bunch of normal distribution around 
    dist = torch.distributions.Normal(obs, 0.1 )
    around_obs = dist.sample()

    logits_around, _ = model(around_obs)

    # get the loss of the state around the observations
    L_S = torch.mean(torch.abs(logits - logits_around))

    # get the loss of the actions around the observations
    L_T = f.action_dist(actions[1:], actions[:-1]).mean()

    # add the loss of the state around the observations to the loss
    loss += L_S
    loss += L_T
    return loss


In [1]:
import torch 
#make random a and b 
a = torch.randn((4,10))
b = torch.randn((4,10))


In [2]:
import constants as c 
act_ranges = torch.tensor(list(c.actuator_ranges.values()))

In [6]:
act_ranges[:,1]-act_ranges[:,0]

tensor([ 9.0000,  9.0000, 24.4000, 24.4000,  1.8000,  9.0000,  9.0000, 24.4000,
        24.4000,  1.8000])

In [3]:
import functions as f 
f.action_dist(a,b)

tensor([[ 0.5389,  2.1878, -0.7917, -2.2485, -2.1279,  1.0671,  0.2755, -0.4149,
          0.1187,  0.7629],
        [-2.1978, -0.8232, -0.4930, -0.2439,  1.6287, -3.3434, -1.7422, -2.2262,
          1.6929, -0.1549],
        [-0.6918, -0.1134,  1.5816,  1.0610,  0.1303, -0.2547,  1.5620,  0.8637,
         -1.3862, -0.3678],
        [ 0.3428, -0.3592, -1.6619,  2.1461, -3.0107, -0.7946, -2.0887, -0.1350,
          1.3064, -2.1819]])


tensor([1.2902, 1.0408, 0.3075, 2.0849])

In [ ]:
#compare the two losses
loss1(train_batch) - loss2(train_batch)

torch.float64


RuntimeError: mat1 and mat2 must have the same dtype